In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from scipy.misc import imresize
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
# Load MNIST dataset
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# split train data in two
X_pr = Xtrain[30001:60000, :, :]
Y_pr = Ytrain[30001:60000]

Xtrain = Xtrain[0:30000, :, :];
Ytrain = Ytrain[0:30000]

# downsample
factor = 0.25

Xtrain_down = np.ones((Xtrain.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtrain.shape[0]):
    Xtrain_down[i, :, :] = imresize(Xtrain[i,:,:], factor)

Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtest.shape[0]):
    Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    
X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(X_pr.shape[0]):
    X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
    
# *** VECTORIZE IMAGES ***
Xtrain_down = Xtrain_down.reshape(Xtrain_down.shape[0], int(xdim*factor)**2).astype('float32') / 255
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

# Categorical labels
Ytrain = np_utils.to_categorical(Ytrain, 10)
Ytest = np_utils.to_categorical(Ytest, 10)
Y_pr = np_utils.to_categorical(Y_pr, 10)

In [94]:
from keras import backend as K
from keras.models import load_model

model = load_model('nn.h5')

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
#functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function

# Testing: layer_outs is a list with 4 data sets
layer_outs = functor([X_pr_down, 1.])


In [134]:
# how many recordings?
nRecordings = 10
# which layers?
layer_outputs = layer_outs
nLayerNeurons = [2, 0, 0, 2]
layerNeurons = list()

# how many samples?
nSamples = X_pr_down.shape[0]/nRecordings
for iLayer in range(len(layer_outputs)):
    print(iLayer)
    layerArray = np.zeros((nRecordings, nLayerNeurons[iLayer]))
    for iRec in range(nRecordings):
        layerArray[iRec, :]= np.random.choice(range(model.layers[iLayer].output_dim), size=nLayerNeurons[iLayer], replace=True)      
    layerNeurons.append(layerArray)   

if len(np.unique(layerNeurons[3]))<model.layers[3].output_dim:
#     #pick #outputs random places and replace them with 1:#outputs.
     layerNeurons[3][np.random.choice(range(nRecordings), size =model.layers[3].output_dim, replace=False), 0] = range(10)   

for iLayer in range(len(layer_outputs)):
    if nLayerNeurons[iLayer]==0:
        continue
    sample_ind =0;
    X_l = layer_outputs[iLayer]
    for iRec in range(nRecordings):
        notObserved = np.setdiff1d(range(model.layers[iLayer].output_dim), layerNeurons[iLayer][iRec, :])
        inds = np.array(range(sample_ind,((iRec+1)*nSamples)))
        X_l[inds[:, None], notObserved] = np.nan;
        sample_ind = sample_ind + nSamples
    if np.sum(nLayerNeurons[0:iLayer])==0:
        data=X_l
    else:
        data = np.append(data, X_l, axis=1)
    
# split the data in nRecordings
# X_rec =  np.random.rand(500, nNeurons)
# nSamples=X_rec.shape[0]/nRecordings

# #nSamples =
# # randomly sample from the first hidden layer
# rand_l1_inds = np.random.choice(range(model.layers[1].output_dim), size=nRecordings, replace=True)
# #randomly sample from the output layer
# rand_o_inds = np.random.choice(range(model.layers[3].output_dim), size=nRecordings, replace=True)

# # make sure you have seen all output units at least once
# if len(np.unique(rand_o_inds))<model.layers[3].output_dim:
#     #pick #outputs random places and replace them with 1:#outputs.
#     rand_o_inds[np.random.choice(range(nRecordings), size =model.layers[3].output_dim, replace=False)] = range(10)
    
# # random permutation of the lines
# X_rec = X_rec[np.random.choice(range(X_rec.shape[0]), size=X_rec.shape[0], replace=False), :]                         
# X_rec_pr = np.nan*np.ones(X_rec.shape)


# sample_ind =0;
# for iRec in range(nRecordings):
#     inds = np.array(range(sample_ind,((iRec+1)*nSamples)))
#     sample_ind = sample_ind + nSamples
#     print(inds)
#     a = np.array([rand_o_inds[iRec], rand_l1_inds[iRec]])
#     X_rec_pr[inds[:, None], a] = X_rec[inds[:, None], a]

0
1
2
3


(29999, 110)